In [2]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import csv
import networkx as nx
from subprocess import check_output
from wordcloud import WordCloud, STOPWORDS
import tensorflow as tf
from IPython.display import YouTubeVideo
plt.style.use('ggplot')

2024-02-26 17:40:54.332011: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [139]:
# Inspect data directory
print(os.listdir("/Users/runzhou/Desktop/youtube/data/yt8m/video/"))

# Explore video-level data
video_lvl_data_path = "/Users/runzhou/Desktop/youtube/data/yt8m/video/train0274.tfrecord"
raw_dataset = tf.data.TFRecordDataset(video_lvl_data_path)


vid_ids = []
labels = []
rgb = []
audio = []

for example in tf.compat.v1.python_io.tf_record_iterator(video_lvl_data_path):
    seq_example = tf.train.Example.FromString(example)
    vid_ids.append(seq_example.features.feature['id'].bytes_list.value[0].decode(encoding='UTF-8'))
    labels.append(seq_example.features.feature['labels'].int64_list.value)
    rgb.append(seq_example.features.feature['mean_rgb'].float_list.value)
    audio.append(seq_example.features.feature['mean_audio'].float_list.value)

print('Number of videos in this tfrecord: ',len(vid_ids))
print ('Number of labels in this tfrecord: ', len (labels))
print('Number of mean RGB in this tfrecord: ', len(rgb))
print('Number of mean audio in this tfrecord: ', len(audio))
print('Picking a youtube video id:', vid_ids[1])

['2_video_train_download_plan.json', 'train0434.tfrecord', 'train0274.tfrecord', 'train0208.tfrecord', 'train0580.tfrecord', 'train2161.tfrecord']
Number of videos in this tfrecord:  1045
Number of labels in this tfrecord:  1045
Number of mean RGB in this tfrecord:  1045
Number of mean audio in this tfrecord:  1045
Picking a youtube video id: zseA


Label:  [14]  |  Video:  https://www.youtube.com/watch?v=200  |  ['Music video']
Label:  [14]  |  Video:  https://www.youtube.com/watch?v=-4Ms2Fc_JoE  |  ['Music video']
Label:  [0, 12, 284, 3754]  |  Video:  https://www.youtube.com/watch?v=ox2hNpnqF9o  |  ['Game', 'Association football', 'GoPro', 'Indoor soccer']
Label:  [15, 18, 67, 140, 1785]  |  Video:  https://www.youtube.com/watch?v=egf8Rc4EkSU  |  ['Animal', 'Pet', 'Dog', 'Puppy', 'Poodle']
Label:  [2, 45, 295]  |  Video:  https://www.youtube.com/watch?v=eg9CRKtCiy8  |  ['Vehicle', 'Train', 'Train station']


In [106]:
vocab

,Index,TrainVideoCount,KnowledgeGraphId,Name,WikiUrl,Vertical1,Vertical2,Vertical3,WikiDescription
0,0,788288,/m/03bt1gh,Game,https://en.wikipedia.org/wiki/Game,Games,NaN,NaN,"A game is structured form of play, usually und..."
1,1,539945,/m/01mw1,Video game,https://en.wikipedia.org/wiki/Video_game,Games,NaN,NaN,A video game is an electronic game that involv...
2,2,415890,/m/07yv9,Vehicle,https://en.wikipedia.org/wiki/Vehicle,Autos & Vehicles,NaN,NaN,A vehicle is a mobile machine that transports ...
3,3,378135,/m/01jddz,Concert,https://en.wikipedia.org/wiki/Concert,Arts & Entertainment,NaN,NaN,A concert is a live music performance in front...
4,4,286532,/m/09jwl,Musician,https://en.wikipedia.org/wiki/Musician,Arts & Entertainment,NaN,NaN,A musician is a person who plays a musical ins...
...,...,...,...,...,...,...,...,...,...
3857,3805,131,/m/02_6gvj,Uncharted: Drake's Fortune,https://en.wikipedia.org/wiki/Uncharted:_Drake...,Games,NaN,NaN,Uncharted: Drake's Fortune is a 2007 action-ad...
3858,3854,130,/m/02kh4w,Injury,https://en.wikipedia.org/wiki/Injury,Health,NaN,NaN,Injury is damage to the body. This may be caus...
3859,3824,130,/m/01hbjs,Look-alike,https://en.wikipedia.org/wiki/Look-alike,(Unknown),NaN,NaN,"A look-alike, or double, is a person who close..."
3860,3844,127,/m/01vzvy,Mortar (masonry),https://en.wikipedia.org/wiki/Mortar_(masonry),Business & Industrial,NaN,NaN,Mortar is a workable paste used to bind buildi...


In [107]:
youtube_urls

['https://www.youtube.com/watch?v=200',
 'https://www.youtube.com/watch?v=-4Ms2Fc_JoE',
 'https://www.youtube.com/watch?v=ox2hNpnqF9o',
 'https://www.youtube.com/watch?v=egf8Rc4EkSU',
 'https://www.youtube.com/watch?v=eg9CRKtCiy8',
 'https://www.youtube.com/watch?v=ejJJLLK9vC8',
 'https://www.youtube.com/watch?v=EQVouZCx5oc',
 'https://www.youtube.com/watch?v=-hyxZkSndfQ',
 'https://www.youtube.com/watch?v=-jxIT8uxhZg',
 'https://www.youtube.com/watch?v=kGNN7sOK32s',
 'https://www.youtube.com/watch?v=kJYcgMOkowY',
 'https://www.youtube.com/watch?v=kKr-SEFwRNg',
 'https://www.youtube.com/watch?v=kQm0-c6pCf4',
 'https://www.youtube.com/watch?v=GJWIDKAfxgo',
 'https://www.youtube.com/watch?v=wg_cHOWgY84',
 'https://www.youtube.com/watch?v=wlXEtTo__PM',
 'https://www.youtube.com/watch?v=wnhxHPZFM4w',
 'https://www.youtube.com/watch?v=mbxcXQmgsbo',
 'https://www.youtube.com/watch?v=mKIPqpKnUOo',
 'https://www.youtube.com/watch?v=JZt4n5O_Gpw',
 'https://www.youtube.com/watch?v=JaC-AqTGYvk',


In [140]:
# Inspect frame-level data directory
print(os.listdir("/Users/runzhou/Desktop/youtube/data/yt8m/frame/"))

import tensorflow as tf

# Explore frame-level data
frame_lvl_data_path = "/Users/runzhou/Desktop/youtube/data/yt8m/frame/train0274.tfrecord"
raw_dataset = tf.data.TFRecordDataset(frame_lvl_data_path)

vid_ids = []
labels = []
rgb = []
audio = []

for example in tf.compat.v1.python_io.tf_record_iterator(frame_lvl_data_path):
    seq_example = tf.train.SequenceExample.FromString(example)
    vid_ids.append(seq_example.context.feature['id'].bytes_list.value[0].decode(encoding='UTF-8'))
    labels.append(seq_example.context.feature['labels'].int64_list.value)

    # Decode RGB features
    rgb_encoded = seq_example.feature_lists.feature_list['rgb'].feature[0].bytes_list.value[0]
    rgb_decoded = tf.io.decode_raw(rgb_encoded, tf.float32)
    rgb.append(rgb_decoded.numpy().tolist())

    # Decode audio features
    audio_encoded = seq_example.feature_lists.feature_list['audio'].feature[0].bytes_list.value[0]
    audio_decoded = tf.io.decode_raw(audio_encoded, tf.float32)
    audio.append(audio_decoded.numpy().tolist())

print('Number of videos in this tfrecord: ',len(vid_ids))
print ('Number of labels in this tfrecord: ', len (labels))
print('Number of mean RGB in this tfrecord: ', len(rgb))
print('Number of mean audio in this tfrecord: ', len(audio))
print('Picking a youtube video id:', vid_ids[1])

['2_frame_train_download_plan.json', 'train0274.tfrecord', 'train0580.tfrecord', 'train2161.tfrecord']
Number of videos in this tfrecord:  1045
Number of labels in this tfrecord:  1045
Number of mean RGB in this tfrecord:  1045
Number of mean audio in this tfrecord:  1045
Picking a youtube video id: WjeA


In [142]:
# Inspect segment-level data directory
print(os.listdir("/Users/runzhou/Desktop/youtube/data/yt8m/segment/"))

# Explore segment-level data
segment_lvl_data_path = "/Users/runzhou/Desktop/youtube/data/yt8m/segment/validate3489.tfrecord"
raw_dataset = tf.data.TFRecordDataset(segment_lvl_data_path)

vid_ids = []
labels = []
rgb = []
audio = []
segment_start_times = []
segment_end_times = []
segment_scores = []
segment_labels = []


for example in tf.compat.v1.python_io.tf_record_iterator(segment_lvl_data_path):
    # Context features
    example_example = tf.train.Example.FromString(example)
    
    vid_ids.append(example_example.features.feature['id'].bytes_list.value[0].decode(encoding='UTF-8'))
    labels.append(example_example.features.feature['labels'].int64_list.value)

    # Segment labels & scores
    segment_start_times.append(example_example.features.feature['segment_start_times'].int64_list.value)
    segment_end_times.append(example_example.features.feature['segment_end_times'].int64_list.value)
    segment_labels.append(example_example.features.feature['segment_labels'].int64_list.value)

    # Frame features
    seq_example = tf.train.SequenceExample.FromString(example)

    # Decode RGB features
    rgb_encoded = seq_example.feature_lists.feature_list['rgb'].feature[0].bytes_list.value[0]
    rgb_decoded = tf.io.decode_raw(rgb_encoded, tf.uint8)
    rgb.append(rgb_decoded.numpy().tolist())

    # Decode audio features
    audio_encoded = seq_example.feature_lists.feature_list['audio'].feature[0].bytes_list.value[0]
    audio_decoded = tf.io.decode_raw(audio_encoded, tf.uint8)
    audio.append(audio_decoded.numpy().tolist())


print('Number of videos in this tfrecord: ',len(vid_ids))
print ('Number of labels in this tfrecord: ', len (labels))
print('Number of mean RGB in this tfrecord: ', len(rgb))
print('Number of mean audio in this tfrecord: ', len(audio))
print('Picking a youtube video id:', vid_ids[1])


['validate3489.tfrecord', '3_frame_validate_download_plan.json']
Number of videos in this tfrecord:  8
Number of labels in this tfrecord:  8
Number of mean RGB in this tfrecord:  8
Number of mean audio in this tfrecord:  8
Picking a youtube video id: Xd4r


In [136]:
#rgb: 8 x 1024
#audio: 8 x 128
len(audio[0])

128

In [143]:
# Translate video ID to Youtube URL
import requests

def translate_video_id(encoded_id):
    # Construct the URL to retrieve the mapping
    url = f"http://data.yt8m.org/2/j/i/{encoded_id[:2]}/{encoded_id}.js"
    response = requests.get(url)

    # Check if the response is successful and has the expected format
    if response.status_code == 200 and 'i("' in response.text:
        # Extract the standard YouTube video ID from the response
        video_id = response.text.split('"')[3]
        return video_id
    else:
        # Log an error or handle it as appropriate
        print(f"Error retrieving video ID for {encoded_id}: {response.text}")
        return '200'

youtube_urls = []
for encoded_id in vid_ids:
    video_id = translate_video_id(encoded_id)
    if video_id:
        youtube_url = f"https://www.youtube.com/watch?v={video_id}"
        youtube_urls.append(youtube_url)

# Print or use the YouTube URLs
for url in youtube_urls:
    print(url)


Error retrieving video ID for Xw4r: <?xml version='1.0' encoding='UTF-8'?><Error><Code>AccessDenied</Code><Message>Access denied.</Message></Error>
Error retrieving video ID for En4r: <?xml version='1.0' encoding='UTF-8'?><Error><Code>AccessDenied</Code><Message>Access denied.</Message></Error>
https://www.youtube.com/watch?v=KYVAv4gq00k
https://www.youtube.com/watch?v=Ph-dQZnHChc
https://www.youtube.com/watch?v=jGve-X-fJug
https://www.youtube.com/watch?v=GnbzWaaIGRY
https://www.youtube.com/watch?v=200
https://www.youtube.com/watch?v=200
https://www.youtube.com/watch?v=DTQpR-AqKnM
https://www.youtube.com/watch?v=tTfptynuWdA


In [153]:
rgb[0]

[0,
 75,
 148,
 37,
 0,
 0,
 74,
 28,
 194,
 118,
 230,
 85,
 58,
 164,
 155,
 195,
 70,
 99,
 63,
 247,
 106,
 225,
 163,
 130,
 110,
 7,
 239,
 131,
 51,
 140,
 128,
 194,
 78,
 72,
 115,
 115,
 112,
 214,
 123,
 81,
 109,
 146,
 102,
 187,
 155,
 125,
 159,
 130,
 164,
 84,
 165,
 112,
 106,
 96,
 93,
 108,
 121,
 120,
 88,
 117,
 99,
 112,
 158,
 114,
 118,
 149,
 178,
 157,
 119,
 94,
 149,
 140,
 137,
 112,
 98,
 81,
 159,
 141,
 120,
 112,
 131,
 120,
 175,
 70,
 112,
 112,
 145,
 159,
 79,
 144,
 205,
 93,
 168,
 137,
 150,
 134,
 142,
 124,
 102,
 166,
 128,
 73,
 128,
 176,
 147,
 128,
 108,
 76,
 116,
 186,
 136,
 120,
 125,
 160,
 154,
 134,
 89,
 78,
 110,
 149,
 146,
 70,
 159,
 102,
 127,
 138,
 154,
 149,
 106,
 169,
 155,
 112,
 200,
 125,
 95,
 110,
 136,
 188,
 137,
 192,
 151,
 136,
 120,
 126,
 141,
 140,
 160,
 111,
 154,
 143,
 140,
 165,
 168,
 124,
 99,
 129,
 204,
 112,
 127,
 116,
 217,
 177,
 136,
 93,
 150,
 131,
 124,
 104,
 104,
 132,
 81,
 87,
 146,
 127

In [204]:
# Read in vocabulary for human-rated mapping of numeric label to text label for the video
vocab = pd.read_csv('./data/vocabulary.csv')

# Create a pandas dataframe to store the segment-rated frame-level contextual data and embeddings
df = pd.DataFrame()

for i, encoded_id in enumerate(vid_ids):
    video_id = translate_video_id(encoded_id)
    if video_id:
        youtube_url = f"https://www.youtube.com/watch?v={video_id}"
    
    text_labels = [vocab[vocab['Index'] == label]['Name'].values[0] for label in labels[i]]
    wiki_descriptions = [vocab[vocab['Index'] == label]['WikiDescription'].values[0] for label in labels[i]]
    text_segment_labels = [vocab[vocab['Index'] == label]['Name'].values[0] for label in segment_labels[i]]
    df = df.append({'labels': labels[i],
                    'text_labels': text_labels,
                    'url': youtube_url,
                    'WikiDescription': wiki_descriptions, 
                    'text_segment_labels': text_segment_labels,
                    'segment_labels': segment_labels[i],
                    'segment_start_times': segment_start_times[i],
                    'segment_end_times': segment_end_times[i],
                    'mean_rgb': rgb[i],
                    'mean_audio': audio[i]},
                    ignore_index=True)

# Display the DataFrame
print(df)

/var/folders/3g/jmc1xdtj4dxgy5q298pc4dyr0000gn/T/ipykernel_32790/4016800596.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'labels': labels[i],
/var/folders/3g/jmc1xdtj4dxgy5q298pc4dyr0000gn/T/ipykernel_32790/4016800596.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'labels': labels[i],
/var/folders/3g/jmc1xdtj4dxgy5q298pc4dyr0000gn/T/ipykernel_32790/4016800596.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'labels': labels[i],
/var/folders/3g/jmc1xdtj4dxgy5q298pc4dyr0000gn/T/ipykernel_32790/4016800596.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'

Error retrieving video ID for Xw4r: <?xml version='1.0' encoding='UTF-8'?><Error><Code>AccessDenied</Code><Message>Access denied.</Message></Error>
Error retrieving video ID for En4r: <?xml version='1.0' encoding='UTF-8'?><Error><Code>AccessDenied</Code><Message>Access denied.</Message></Error>


/var/folders/3g/jmc1xdtj4dxgy5q298pc4dyr0000gn/T/ipykernel_32790/4016800596.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'labels': labels[i],
/var/folders/3g/jmc1xdtj4dxgy5q298pc4dyr0000gn/T/ipykernel_32790/4016800596.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'labels': labels[i],
/var/folders/3g/jmc1xdtj4dxgy5q298pc4dyr0000gn/T/ipykernel_32790/4016800596.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'labels': labels[i],


                   labels                                   text_labels  \
0       [2, 76, 281, 446]             [Vehicle, Boat, Motorboat, Yacht]   
1                   [191]                                       [Hotel]   
2  [11, 1366, 1684, 1872]    [Food, Grocery store, Supermarket, Retail]   
3        [2, 7, 116, 235]    [Vehicle, Car, Sport utility vehicle, nan]   
4             [461, 1325]                    [Printer (computing), Ink]   
5                   [607]                                  [Board game]   
6       [0, 1, 234, 1303]  [Game, Video game, Super Smash Bros., Yoshi]   
7                [17, 19]                          [Motorsport, Racing]   

                                           url  \
0  https://www.youtube.com/watch?v=KYVAv4gq00k   
1  https://www.youtube.com/watch?v=Ph-dQZnHChc   
2  https://www.youtube.com/watch?v=jGve-X-fJug   
3  https://www.youtube.com/watch?v=GnbzWaaIGRY   
4          https://www.youtube.com/watch?v=200   
5          https://www.y

/var/folders/3g/jmc1xdtj4dxgy5q298pc4dyr0000gn/T/ipykernel_32790/4016800596.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'labels': labels[i],


In [205]:
df.head(5)

,labels,text_labels,url,WikiDescription,text_segment_labels,segment_labels,segment_start_times,segment_end_times,mean_rgb,mean_audio
0,"[2, 76, 281, 446]","[Vehicle, Boat, Motorboat, Yacht]",https://www.youtube.com/watch?v=KYVAv4gq00k,[A vehicle is a mobile machine that transports...,"[Luxury yacht, Luxury yacht, Luxury yacht, Lux...","[1314, 1314, 1314, 1314, 1314]","[50, 165, 230, 115, 210]","[55, 170, 235, 120, 215]","[0, 75, 148, 37, 0, 0, 74, 28, 194, 118, 230, ...","[175, 43, 116, 109, 154, 93, 75, 198, 172, 171..."
1,[191],[Hotel],https://www.youtube.com/watch?v=Ph-dQZnHChc,[A hotel is an establishment that provides pai...,"[Hotel, Hotel, Hotel]","[191, 191, 191]","[200, 255, 230]","[205, 260, 235]","[89, 195, 86, 213, 96, 100, 151, 183, 103, 151...","[163, 55, 116, 98, 91, 183, 151, 157, 177, 117..."
2,"[11, 1366, 1684, 1872]","[Food, Grocery store, Supermarket, Retail]",https://www.youtube.com/watch?v=jGve-X-fJug,[Food is any substance consumed to provide nut...,"[Grocery store, Grocery store, Grocery store, ...","[1366, 1366, 1366, 1366, 1366]","[85, 220, 230, 125, 45]","[90, 225, 235, 130, 50]","[0, 72, 173, 30, 0, 0, 40, 33, 199, 132, 199, ...","[175, 47, 132, 85, 169, 168, 156, 167, 157, 13..."
3,"[2, 7, 116, 235]","[Vehicle, Car, Sport utility vehicle, nan]",https://www.youtube.com/watch?v=GnbzWaaIGRY,[A vehicle is a mobile machine that transports...,"[Sport utility vehicle, Sport utility vehicle,...","[116, 116, 116, 235, 116, 235, 235, 116, 235, ...","[265, 220, 65, 180, 105, 230, 105, 85, 135, 165]","[270, 225, 70, 185, 110, 235, 110, 90, 140, 170]","[95, 203, 71, 171, 124, 165, 110, 172, 123, 17...","[157, 37, 131, 92, 127, 93, 194, 217, 140, 103..."
4,"[461, 1325]","[Printer (computing), Ink]",https://www.youtube.com/watch?v=200,"[In computing, a printer is a peripheral which...","[Ink, Ink, Ink, Ink, Ink]","[1325, 1325, 1325, 1325, 1325]","[50, 45, 85, 35, 110]","[55, 50, 90, 40, 115]","[86, 33, 92, 131, 206, 63, 92, 178, 43, 139, 5...","[133, 86, 84, 137, 155, 55, 95, 83, 153, 123, ..."


#### Build a generic RAG system

In [190]:
#!sudo apt-get install build-essential libssl-dev python3-dev
#!python -m pip install --upgrade pip setuptools wheel
#!python -m pip install grpcio


#!python -m pip install chromadb

#!python --version



  Using cached chromadb-0.4.24-py3-none-any.whl.metadata (7.3 kB)
  Using cached build-1.1.1-py3-none-any.whl.metadata (4.2 kB)
  Using cached chroma_hnswlib-0.7.3-cp310-cp310-macosx_10_9_x86_64.whl.metadata (252 bytes)
  Using cached fastapi-0.110.0-py3-none-any.whl.metadata (25 kB)
  Using cached uvicorn-0.27.1-py3-none-any.whl.metadata (6.3 kB)
  Using cached posthog-3.4.2-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached pulsar_client-3.4.0-cp310-cp310-macosx_10_15_universal2.whl.metadata (1.0 kB)
  Using cached onnxruntime-1.16.3-cp310-cp310-macosx_10_15_x86_64.whl.metadata (4.3 kB)
  Using cached opentelemetry_api-1.23.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.23.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached opentelemetry_instrumentation_fastapi-0.44b0-py3-none-any.whl.metadata (2.3 kB)
  Using cached opentelemetry_sdk-1.23.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached PyPika-0.48.9-py2.py3-none-any.whl
  Using ca

In [206]:
df.head(10)

,labels,text_labels,url,WikiDescription,text_segment_labels,segment_labels,segment_start_times,segment_end_times,mean_rgb,mean_audio
0,"[2, 76, 281, 446]","[Vehicle, Boat, Motorboat, Yacht]",https://www.youtube.com/watch?v=KYVAv4gq00k,[A vehicle is a mobile machine that transports...,"[Luxury yacht, Luxury yacht, Luxury yacht, Lux...","[1314, 1314, 1314, 1314, 1314]","[50, 165, 230, 115, 210]","[55, 170, 235, 120, 215]","[0, 75, 148, 37, 0, 0, 74, 28, 194, 118, 230, ...","[175, 43, 116, 109, 154, 93, 75, 198, 172, 171..."
1,[191],[Hotel],https://www.youtube.com/watch?v=Ph-dQZnHChc,[A hotel is an establishment that provides pai...,"[Hotel, Hotel, Hotel]","[191, 191, 191]","[200, 255, 230]","[205, 260, 235]","[89, 195, 86, 213, 96, 100, 151, 183, 103, 151...","[163, 55, 116, 98, 91, 183, 151, 157, 177, 117..."
2,"[11, 1366, 1684, 1872]","[Food, Grocery store, Supermarket, Retail]",https://www.youtube.com/watch?v=jGve-X-fJug,[Food is any substance consumed to provide nut...,"[Grocery store, Grocery store, Grocery store, ...","[1366, 1366, 1366, 1366, 1366]","[85, 220, 230, 125, 45]","[90, 225, 235, 130, 50]","[0, 72, 173, 30, 0, 0, 40, 33, 199, 132, 199, ...","[175, 47, 132, 85, 169, 168, 156, 167, 157, 13..."
3,"[2, 7, 116, 235]","[Vehicle, Car, Sport utility vehicle, nan]",https://www.youtube.com/watch?v=GnbzWaaIGRY,[A vehicle is a mobile machine that transports...,"[Sport utility vehicle, Sport utility vehicle,...","[116, 116, 116, 235, 116, 235, 235, 116, 235, ...","[265, 220, 65, 180, 105, 230, 105, 85, 135, 165]","[270, 225, 70, 185, 110, 235, 110, 90, 140, 170]","[95, 203, 71, 171, 124, 165, 110, 172, 123, 17...","[157, 37, 131, 92, 127, 93, 194, 217, 140, 103..."
4,"[461, 1325]","[Printer (computing), Ink]",https://www.youtube.com/watch?v=200,"[In computing, a printer is a peripheral which...","[Ink, Ink, Ink, Ink, Ink]","[1325, 1325, 1325, 1325, 1325]","[50, 45, 85, 35, 110]","[55, 50, 90, 40, 115]","[86, 33, 92, 131, 206, 63, 92, 178, 43, 139, 5...","[133, 86, 84, 137, 155, 55, 95, 83, 153, 123, ..."
5,[607],[Board game],https://www.youtube.com/watch?v=200,[A board game is a tabletop game that involves...,"[Board game, Board game, Board game, Board gam...","[607, 607, 607, 607, 607]","[95, 215, 140, 60, 145]","[100, 220, 145, 65, 150]","[116, 161, 174, 145, 99, 88, 64, 117, 147, 104...","[105, 70, 109, 126, 116, 29, 122, 114, 181, 18..."
6,"[0, 1, 234, 1303]","[Game, Video game, Super Smash Bros., Yoshi]",https://www.youtube.com/watch?v=DTQpR-AqKnM,"[A game is structured form of play, usually un...","[Yoshi, Yoshi, Yoshi, Yoshi, Yoshi]","[1303, 1303, 1303, 1303, 1303]","[185, 45, 190, 105, 195]","[190, 50, 195, 110, 200]","[181, 16, 82, 162, 190, 255, 88, 28, 202, 213,...","[156, 218, 194, 148, 121, 42, 8, 73, 68, 35, 9..."
7,"[17, 19]","[Motorsport, Racing]",https://www.youtube.com/watch?v=tTfptynuWdA,[Motorsport or motorsports is a global term us...,"[Running, Running, Running, Running, Running]","[210, 210, 210, 210, 210]","[40, 65, 80, 30, 55]","[45, 70, 85, 35, 60]","[120, 193, 70, 224, 104, 170, 113, 125, 118, 1...","[65, 70, 167, 165, 166, 107, 173, 41, 113, 238..."


In [224]:
import chromadb

# Initialize the ChromaDB client
chroma_client = chromadb.Client()

#Create a collections to store your embeddings, documents, and any additional metadata. 
collection = chroma_client.get_or_create_collection(name='rgb_collection')

# Store embeddings in ChromaDB
for i, row in df.iterrows():
    # Create a unique ID for each embedding
    id = f'{i}'

    # Get info of each row
    metadatas_dict = {
    #'labels': row['labels'],
    #'text_labels': row['text_labels'],
    #'segment_labels': row['segment_labels'],
    #'text_segment_labels': row['text_segment_labels'],
    #'segment_start_times': row['segment_start_times'],
    #'segment_end_times': row['segment_end_times'],
    'url': row['url'],
    'wiki': row['WikiDescription'][0]}

    # Store the embedding in ChromaDB
    collection.add(
        embeddings=row['mean_rgb'],
        metadatas=metadatas_dict,
        ids=id
    )

print('Embeddings stored in ChromaDB successfully.')


Embeddings stored in ChromaDB successfully.


In [229]:
rgb[5]

[116,
 161,
 174,
 145,
 99,
 88,
 64,
 117,
 147,
 104,
 143,
 120,
 172,
 151,
 83,
 203,
 91,
 0,
 166,
 186,
 76,
 155,
 137,
 153,
 140,
 210,
 114,
 207,
 44,
 104,
 206,
 144,
 66,
 94,
 87,
 145,
 249,
 98,
 157,
 113,
 147,
 93,
 205,
 58,
 126,
 98,
 176,
 123,
 94,
 130,
 255,
 170,
 125,
 9,
 100,
 187,
 99,
 161,
 157,
 160,
 116,
 92,
 87,
 179,
 100,
 137,
 0,
 116,
 0,
 49,
 195,
 99,
 105,
 191,
 48,
 96,
 255,
 134,
 58,
 133,
 255,
 60,
 214,
 106,
 190,
 149,
 208,
 111,
 84,
 45,
 26,
 255,
 58,
 119,
 111,
 139,
 153,
 135,
 150,
 66,
 123,
 123,
 61,
 116,
 138,
 165,
 0,
 102,
 118,
 61,
 229,
 91,
 175,
 124,
 133,
 77,
 32,
 212,
 190,
 230,
 159,
 117,
 171,
 125,
 76,
 0,
 106,
 108,
 21,
 196,
 88,
 146,
 99,
 190,
 70,
 213,
 83,
 150,
 190,
 229,
 216,
 195,
 157,
 52,
 110,
 205,
 37,
 13,
 88,
 151,
 0,
 113,
 202,
 106,
 255,
 2,
 123,
 134,
 190,
 122,
 163,
 47,
 128,
 190,
 115,
 161,
 60,
 57,
 187,
 105,
 114,
 151,
 174,
 83,
 167,
 151,
 160,
 9

In [227]:
# get items from a collection
collection.peek()



{'ids': ['0', '1', '2', '3', '4', '5', '6', '7'],
 'embeddings': [[0.0,
   75.0,
   148.0,
   37.0,
   0.0,
   0.0,
   74.0,
   28.0,
   194.0,
   118.0,
   230.0,
   85.0,
   58.0,
   164.0,
   155.0,
   195.0,
   70.0,
   99.0,
   63.0,
   247.0,
   106.0,
   225.0,
   163.0,
   130.0,
   110.0,
   7.0,
   239.0,
   131.0,
   51.0,
   140.0,
   128.0,
   194.0,
   78.0,
   72.0,
   115.0,
   115.0,
   112.0,
   214.0,
   123.0,
   81.0,
   109.0,
   146.0,
   102.0,
   187.0,
   155.0,
   125.0,
   159.0,
   130.0,
   164.0,
   84.0,
   165.0,
   112.0,
   106.0,
   96.0,
   93.0,
   108.0,
   121.0,
   120.0,
   88.0,
   117.0,
   99.0,
   112.0,
   158.0,
   114.0,
   118.0,
   149.0,
   178.0,
   157.0,
   119.0,
   94.0,
   149.0,
   140.0,
   137.0,
   112.0,
   98.0,
   81.0,
   159.0,
   141.0,
   120.0,
   112.0,
   131.0,
   120.0,
   175.0,
   70.0,
   112.0,
   112.0,
   145.0,
   159.0,
   79.0,
   144.0,
   205.0,
   93.0,
   168.0,
   137.0,
   150.0,
   134.0,
   142.0

In [237]:
# do nearest neighbor search to find similar embeddings or documents, supports filtering
collection.query(
    query_embeddings=rgb[3],
    n_results=2
    #where={"style": "style2"}
)

{'ids': [['3', '2']],
 'distances': [[0.0, 3891342.0]],
 'metadatas': [[{'url': 'https://www.youtube.com/watch?v=GnbzWaaIGRY',
    'wiki': 'A vehicle is a mobile machine that transports people or cargo. Typical vehicles include wagons, bicycles, motor vehicles, railed vehicles, watercraft, aircraft and spacecraft. Land vehicles are classified broadly by what is used to apply steering and drive forces against the ground: wheeled, tracked, railed or skied. ISO 3833-1977 is the standard, also internationally used in legislation, for road vehicles types, terms and definitions.'},
   {'url': 'https://www.youtube.com/watch?v=jGve-X-fJug',
    'wiki': "Food is any substance consumed to provide nutritional support for the body. It is usually of plant or animal origin, and contains essential nutrients, such as carbohydrates, fats, proteins, vitamins, or minerals. The substance is ingested by an organism and assimilated by the organism's cells to provide energy, maintain life, or stimulate growt

In [233]:
df.iloc[6]

labels                                                 [0, 1, 234, 1303]
text_labels                 [Game, Video game, Super Smash Bros., Yoshi]
url                          https://www.youtube.com/watch?v=DTQpR-AqKnM
WikiDescription        [A game is structured form of play, usually un...
text_segment_labels                  [Yoshi, Yoshi, Yoshi, Yoshi, Yoshi]
segment_labels                            [1303, 1303, 1303, 1303, 1303]
segment_start_times                             [185, 45, 190, 105, 195]
segment_end_times                               [190, 50, 195, 110, 200]
mean_rgb               [181, 16, 82, 162, 190, 255, 88, 28, 202, 213,...
mean_audio             [156, 218, 194, 148, 121, 42, 8, 73, 68, 35, 9...
Name: 6, dtype: object

In [234]:
df.iloc[2]

labels                                            [11, 1366, 1684, 1872]
text_labels                   [Food, Grocery store, Supermarket, Retail]
url                          https://www.youtube.com/watch?v=jGve-X-fJug
WikiDescription        [Food is any substance consumed to provide nut...
text_segment_labels    [Grocery store, Grocery store, Grocery store, ...
segment_labels                            [1366, 1366, 1366, 1366, 1366]
segment_start_times                              [85, 220, 230, 125, 45]
segment_end_times                                [90, 225, 235, 130, 50]
mean_rgb               [0, 72, 173, 30, 0, 0, 40, 33, 199, 132, 199, ...
mean_audio             [175, 47, 132, 85, 169, 168, 156, 167, 157, 13...
Name: 2, dtype: object